# Entity Relationship Diagram (erd_v2)


Actualizamos el diagrama de entidad relación agregando algunos atributos a las entidades `Book` y `Author`.


![erd_02](https://raw.githubusercontent.com/maq-miguel-quinteros/00_apuntes/main/03_java-python/API%20RESTful%20Java%20and%20SpringBoot/wip/img/erd_02.png)


# New `Author` entity, repository & service


## Entity


Dentro del `package models` creamos una nueva clase llamada `Author` y la configuramos de forma similar a la clase `Book`.


In [ ]:
@Entity
public class Author {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String name;
    private String lastName;
    private LocalDate birthdate;
    private LocalDate deathDate;  

    public Author() {
    }

    public Author(String name, String lastName, LocalDate birthdate, LocalDate deathDate) {
        this.name = name;
        this.lastName = lastName;
        this.birthdate = birthdate;
        this.deathDate = deathDate;
    }

    // GETTERS Y SETTERS EXCEPTO EL setId
}

## Respostory & service


### Repository


Dentro de la `package repositories` creamos la interface llamada `AuthorRepository`.


In [ ]:
@RepositoryRestResource
public interface AuthorRepository extends JpaRepository <Author, Long> {
}

### DTO


Dentro de la `package dtos` creamos la clase llamada `AuthorDTO`


In [ ]:
public class AuthorDTO {

    private Long id;
    private String name;
    private String lastName;
    private LocalDate birthdate;
    private LocalDate deathDate;

    public AuthorDTO() {
    }
    public AuthorDTO(Author author) {
        this.id = author.getId();
        this.name = author.getName();
        this.lastName = author.getLastName();
        this.birthdate = author.getBirthdate();
        this.deathDate = author.getDeathDate();
    }

    // SOLO GETTERS
}

### Service


Dentro de la `package services` creamos la interface llamada `AuthorService`.


In [ ]:
public interface AuthorService {

    // CREATE
    void create(Author author);

    // READ
    List<AuthorDTO> readAll();
}

#### Implement


Dentro del `package implement` creamos la clase llamada `AuthorServiceImplement`.


In [ ]:
@Service
public class AuthorServiceImplement implements AuthorService {

    @Autowired
    AuthorRepository authorRepository;
    
    @Override
    public void create(Author author) {
        authorRepository.save(author);
    }

    @Override
    public List<AuthorDTO> readAll() {
        return authorRepository.findAll().stream()
                .map(author -> new AuthorDTO(author))
                .collect(Collectors.toList());
    }
}

# Update `Book` entity


Actualizamos la definición de la clase `Book` agregando el atributo `pagesNumber` de tipo `int`, sumando el mismo al constructor de la clase y creando su `getter` y `setter`.


In [ ]:
@Entity
public class Book {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String isbn;
    private String title;
    private LocalDate date;
    @Lob
    private String synopsis;
    private int pagesNumber;

    public Book() {}
    public Book(String isbn, String title, LocalDate date ,String synopsis, int pagesNumber) {
        this.isbn = isbn;
        this.title = title;
        this.date = date;
        this.synopsis = synopsis;
        this.pagesNumber = pagesNumber;
    }

    // GETTERS & SETTERS EXCEPTO setId

}

# Entity relations


Como podemos observar en el diagrama de entidad relación erd_v2, del comienzo del documento, y tomando en cuenta la lógica con la que desarrollamos el diagrama, que nos dice que un libro `Book` puede tener uno o mas autores `Author`, estableciendo una relación de uno a muchos (`1...N`) de libro a autor, y que un autor lo puede ser de uno o muchos libros, estableciendo el mismo tipo de relación (`1...N`) de autor a libro, nos vemos en la necesidad de crear una entidad intermedia, que denominamos `BookAuthor`, para establecer la relación entre libros y autores, y normalizar estas tablas en la base de datos.


## `BookAuthor` entity


Dentro del `package models` creamos una nueva clase llamada `BookAuthor` y la configuramos de la siguiente manera.

- `@ManyToOne`: mediante la `annotation @ManyToOne` indicamos que la entidad generada a partir de la clase `BookAuthor` va a tener una relación de muchos a uno (`N...1`) con el tipo del atributo que está debajo de la `annotation`, que en ejemplo son `book` y `author`, quiere decir que, mediante esta `annotation`, estamos configurando la relación de muchos a uno entre la entidad `BookAuthor` con respecto a la entidad `Book`, por un lado, y la entidad `Author` por el otro.

- `fetch = FetchType.EAGER`: mediante el parámetro `fetch`, que pasamos a la `annotation`, indicamos la forma en que va a ser cargada la entidad. Al parámetro le asignamos el valor `FetchType.EAGER` que indica que la relación debe ser cargada al momento de cargar la entidad.
- `@JoinColumn`(name = "book_id"): mediante la `annotation @JoinColumn` indicamos que el atributo que está debajo va a requerir de un `Join` para poder acceder al mismo, es decir que el atributo viene dado por una relación con otra entidad, en este caso por la entidad `Book`, y la columna en la base de datos correspondiente a este atributo se configura a partir de esa relación.
- `name = "book_id"` y `name = "book_id"`: mediante el parámetro `name` que pasamos a la `annotation` indicamos el nombre de la columna en la base de datos para los atributos de tipo `Book` y tipo `Author`, que en el ejemplo va a ser `"book_id"` y `author_id`.
- No generamos el constructor con parámetros para la clase por que el único atributo propio de la misma, `id`, se genera de forma automática por la `annotation @GeneratedValue`. Los atributos `book` y `author` pertenecen a la relación con las entidades de `Book` y `Author` por lo que no se le asignan atributos desde un constructor. Si la clase tuviese atributos propios además de `id` correspondería generar el constructor.


In [ ]:
@Entity
public class BookAuthor {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;

    @ManyToOne(fetch = FetchType.EAGER)
    @JoinColumn(name = "book_id")
    private Book book;

    @ManyToOne(fetch = FetchType.EAGER)
    @JoinColumn(name = "author_id")
    private Author author;

    public BookAuthor() {
    }
    // GETTERS & SETTERS EXCEPTO EL setId
}

## `BookAuthor` repository & service


### Repository


Dentro de la `package repositories` creamos la interface llamada `BookAuthorRepository`.


In [ ]:
@RepositoryRestResource
public interface BookAuthorRepository extends JpaRepository <BookAuthor, Long> {
}

### DTO


Dentro de la `package dtos` creamos la clase llamada `BookAuthorDTO` y la configuramos de la siguiente manera.

- Los atributos del dto no tienen que ser, necesariamente, los atributos de la clase a la que refiere el mismo. La definición de estos atributos va a depender de cuales sean los datos que necesitemos pasar al frontend, esto quiere decir que usamos el dto como su nombre indica, data transfer object, para pasar mediante un objeto los datos que queremos pasar.

- La clase del dto tiene atributos propios del dto como `author_name`, `author_lastName`, `author_birthdate` y `author_deathDate` que van a ser asignados en base al atributo de tipo `Author` del objeto `BookAuthor` que pasamos a la constructor del dto.
- `bookAuthor.getAuthor().getName()`: mediante este método `getAuthor()`, del objeto `bookAuthor`, traemos del objeto de tipo `Author` que este tiene, y mediante el método `getName()` del objeto de tipo `Author` traemos el atributo `name` de este.


In [ ]:
public class BookAuthorDTO {

    private Long id;
    private String author_name;
    private String author_lastName;
    private LocalDate author_birthdate;
    private LocalDate author_deathDate;

    public BookAuthorDTO() {
    }

    public BookAuthorDTO(BookAuthor bookAuthor) {
        this.id = bookAuthor.getId();
        this.author_name = bookAuthor.getAuthor().getName();
        this.author_lastName = bookAuthor.getAuthor().getLastName();
        this.author_birthdate = bookAuthor.getAuthor().getBirthdate();
        this.author_deathDate = bookAuthor.getAuthor().getDeathDate();
    }

    // SOLO GETTERS
}

### Service


Dentro de la `package services` creamos la interface llamada `BookAuthorService`.


In [ ]:
public interface BookAuthorService {

    // CREATE
    void create(BookAuthor bookAuthor);

}

#### Implement


Dentro del `package implement` creamos la clase llamada `BookAuthorServiceImplement`.


In [ ]:
@Service
public class BookAuthorServiceImplement implements BookAuthorService {

    @Autowired
    BookAuthorRepository bookAuthorRepository;

    @Override
    public void create(BookAuthor bookAuthor) {
        bookAuthorRepository.save(bookAuthor);
    }
}

## Update `Book` entity


Actualizamos la definición de la clase `Book` de la siguiente manera.

- `@OneToMany`: mediante la `annotation @OneToMany` indicamos que la entidad generada a partir de la clase `Book` va a tener una relación de uno a muchos (`1...N`) con el tipo del atributo que está debajo de la `annotation`, que en ejemplo es `BookAuthor`, quiere decir que, mediante esta `annotation`, estamos configurando la relación de uno a muchos entre la entidad `Book` con respecto a la entidad `BookAuthor`.

- `mappedBy = "book"`: mediante el parámetro `mappedBy`, que pasamos a la `annotation`, indicamos el nombre de la columna que, en la base de datos, van a tener las entradas correspondientes al listado de autores del libro, listado que puede tener uno o muchos autores.
- `Set<BookAuthor> authors`: al indicar que el tipo de datos del atributo `author` es `Set<BookAuthor>` indicamos que el mismo va a ser un `Set`, un tipo de datos que representa un listado, de objetos de tipo `BookAuthor`.
- `new HashSet<>()`: mediante la interface `HashSet` generamos los objetos para el listado de tipo `Set`.
- El atributo `authors` pertenece a la relación con la entidad `BookAuthor` por lo que no se le asignan valores desde un constructor.
- `addBookAuthor(BookAuthor bookAuthor)`: generamos el método `addBookAuthor`, que recibe como parámetro un objeto de tipo `BookAuthor`, mediante el cual vamos a sumar autores al listado de estos que se corresponde con el atributo `authors`.
- `bookAuthor.setBook(this)`: llamamos al método `setBook`, del objeto `BookAuthor` que pasamos por parámetro, y mediante el parámetro `this` indicamos que se asigne el valor del libro actual, con todos sus atributos, al atributo de tipo `book` del objeto de tipo `BookAuthor`.
- `authors.add(bookAuthor)`: el método `add`, que hereda el atributo `authors` por ser de tipo `Set`, agrega un nuevo objeto de tipo `BookAuthor` al listado de autores del libro.


In [ ]:
@Entity
public class Book {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String isbn;
    private String title;
    private LocalDate date;
    @Lob
    private String synopsis;
    private int pagesNumber;
    @OneToMany(mappedBy = "book", fetch = FetchType.EAGER)
    private Set<BookAuthor> authors = new HashSet<>();

    public Book() {}
    public Book(String isbn, String title, LocalDate date ,String synopsis, int pagesNumber) {
        this.isbn = isbn;
        this.title = title;
        this.date = date;
        this.synopsis = synopsis;
        this.pagesNumber = pagesNumber;
    }

    public void addBookAuthor(BookAuthor bookAuthor){
        bookAuthor.setBook(this);
        authors.add(bookAuthor);
    }

    // GETTERS & SETTERS EXCEPTO setId
}

## Update `Author` entity


Actualizamos la definición de la clase `Author` de la misma forma que la clase `Book`, creando un atributo llamado `books` definido como un listado `Set` de objetos `BookAuthor`.


In [ ]:
@Entity
public class Author {

    @Id
    @GeneratedValue(strategy = GenerationType.AUTO, generator = "native")
    @GenericGenerator(name = "native")
    private Long id;
    private String name;
    private String lastName;
    private LocalDate birthdate;
    private LocalDate deathDate;
    @OneToMany(mappedBy = "author", fetch = FetchType.EAGER)
    private Set<BookAuthor> books = new HashSet<>();

    public Author() { }
    public Author(String name, String lastName, LocalDate birthdate, LocalDate deathDate) {
        this.name = name;
        this.lastName = lastName;
        this.birthdate = birthdate;
        this.deathDate = deathDate;
    }

    public void addBookAuthor(BookAuthor bookAuthor){
        bookAuthor.setAuthor(this);
        books.add(bookAuthor);
    }

    // 
}

## Update `BookDTO` class


Actualizamos la definición de la clase `BookDTO` de la siguiente manera.

- `Set<BookAuthorDTO> authors`: al indicar que el tipo de datos del atributo `authors` es `Set<BookAuthorDTO>` indicamos que el mismo va a ser un `Set`, un tipo de datos que representa un listado, de objetos de tipo `BookAuthorDTO`.

- `book.getAuthors()`: mediante el método `getAuthors` del objeto de tipo `Book`, que pasamos por parámetro al constructor del DTO, vamos a generar el listado `Set` de objetos de tipo de objetos de tipo `BookAuthorDTO`. Como la clase `BookAuthorDTO` ya está configurada para devolver los datos que queremos del objeto BookAuthor, que son los datos de el o los autores del libro, es que utilizamos este para generar el listado de autores del atributo `authors`.


In [ ]:
public class BookDTO {

    private Long id;
    private String isbn;
    private String title;
    private LocalDate date;
    private String synopsis;
    private int pagesNumber;
    private Set<BookAuthorDTO> authors;

    public BookDTO() {
    }
    public BookDTO(Book book) {
        this.id = book.getId();
        this.isbn = book.getIsbn();
        this.title = book.getTitle();
        this.date = book.getDate();
        this.synopsis = book.getSynopsis();
        this.pagesNumber = book.getPagesNumber();

        this.authors = book.getAuthors().stream()
                .map(bookAuthor -> new BookAuthorDTO(bookAuthor))
                .collect(Collectors.toSet());
    }

    // SOLO LOS GETTERS
}

# H2 database test


Actualizamos la clase principal del proyecto, en este caso llamada `StockApplication`, para crear las tablas y registros de prueba mediante la base de datos de H2.

- `bookAuthor02 = new BookAuthor()`: creamos un nuevo objeto de tipo `BookAutor`. Esta clase, de momento, no tiene atributos exclusivos, solo el `id` que se genera en forma automática y los `Set` de `Book` y `Author`, por esto no pasamos parámetros al constructor.

- `addBookAuthor`: después de crear el objeto de tipo `BookAuthor` utilizamos el método `addBookAuthor`, presente en el objeto de tipo `Book` y el objeto de tipo `Author`, para sumar a sus respectivos listados `Set` este objeto. Ese mismo método, a su vez, establece la relación del objeto de tipo `Book`m o `Author` con respecto al objeto `BookAuthor`, en la base de datos, mediante el método add presente en la definición de `addBookAuthor`.

- En el tercer ejemplo creamos un objeto de tipo `Book`, un libro, y dos objetos de tipo `Author`, dos autores. Después creamos dos objetos de tipo `BookAuthor`, entendiendo por esto que vamos a sumar dos filas a la tabla de la entidad `book_author` en la base de datos. Sabemos que el libro del ejemplo 3 tiene dos autores, los dos que creamos, por ende, al llamar al método `addBookAuthor`, en el caso del libro, lo hacemos dos veces para el mismo libro, dos veces para el objeto `book03` pasando como parámetro los objetos `bookAuthor03` y `bookAuthor04`. En el caso de los autores, sabemos que ambos autores lo son del mismo libro, del libro del objeto `book03`, al que relacionamos mediante el método `addBookAuthor` con los dos objetos de tipo `BookAuthor`. Sabiendo eso vamos a establecer la relación del autor con la entidad `book_autor`, de la base de datos, mediante el método `addBookAuthor` del objeto `author03`, con respecto al objeto bookAuthor03, y mediante el método `addBookAuthor` del objeto `author04`, con respecto al objeto `bookAuthor04`.

- En la tabla `book_author`, de la base de datos, vamos a tener dos filas donde el valor de la columna `book_id` va a ser el mismo, va a ser el valor del atributo `id` del objeto `book03`. Mientras que, para estas dos filas, vamos a tener dos valores distintos en la columna `author_id`, estos valores van a ser los de los atributos `id` de los objetos `author03` y `author04`.


In [ ]:
@SpringBootApplication
public class StockApplication {

    @Autowired
    BookService bookService;
    @Autowired
    AuthorService authorService;

    @Autowired
    BookAuthorService bookAuthorService;

    public static void main(String[] args) {
        SpringApplication.run(StockApplication.class, args);
    }

    @Bean
    public CommandLineRunner initData() {
        return (args -> {
            // CREAMOS UN NUEVO LIBRO MEDIANTE LA CLASE Book
            Book book01 = new Book("9788466619523", "A través del tiempo", LocalDate.of(2018, 11, 22),
                    "El doctor Brian Weiss, analiza en este libro la capacidad de curación de la terapia de regresión a vidas pasadas.",
                    240);
            bookService.create(book01);

            // CREAMOS UN NUEVO AUTOR mediante la clase Author
            Author author01 = new Author("Brian", "Weiss", LocalDate.of(1994, 11, 6), null);
            authorService.create(author01);

            // CREAMOS LA RELACIÓN ENTRE LIBRO Y AUTOR MEDIANTE LA CLASE BookAutor
            BookAuthor bookAuthor01 = new BookAuthor();
            book01.addBookAuthor(bookAuthor01);
            author01.addBookAuthor(bookAuthor01);
            bookAuthorService.create(bookAuthor01);

            /** *********************************************************** */

            Book book02 = new Book("9789875505681", "Los Miserables", LocalDate.of(2010, 07, 02),
                    "Es la historia de Jean Valjean, un convicto que estuvo injustamente encarcelado por 19 años por haberse robado una rebanada de pan. Al ser liberado de su injusta condena, Valjean trata de escapar de su pasado, lleno de maldad y depravación, para vivir una vida digna y honesta. Sin embargo, esto se ve truncado al ser reconocido por el inspector Javert, quien lo persigue obsesionadamente para enviarlo de nuevo a prisión. Esta persecución consume la vida de ambos hombres, terminando en un inesperado desenlace.",
                    1090);
            bookService.create(book02);

            Author author02 = new Author("Victor", "Hugo", LocalDate.of(1802, 2, 26), LocalDate.of(1885, 5, 22));
            authorService.create(author02);

            BookAuthor bookAuthor02 = new BookAuthor();
            book02.addBookAuthor(bookAuthor02);
            author02.addBookAuthor(bookAuthor02);
            bookAuthorService.create(bookAuthor02);

            /** *********************************************************** */

            Book book03 = new Book("9788416788712", "La Bolsa o la Vida", LocalDate.of(1992, 9, 01),
                    "El libro definitivo para dejar de ser esclavo del dinero y mejorar tu vida. Hazte las siguientes preguntas: ¿Tienes bastante dinero? ¿Pasas suficiente tiempo con tu familia y amigos? ¿Vuelves a casa del trabajo lleno de energía? ¿Tienes una vida completa y estás satisfecho? Si has respondido que no a alguna de estas preguntas, este libro es para ti. Vicki Robin ofrece en La bolsa o la vida un programa de nueve pasos con el que descubrirás el valor real de tu tiempo, transformarás tu relación con el dinero, aprenderás a simplificar y a vivir bien gastando menos y, además, recuperarás el control de tu vida. «Este libro te cambiará la vida.» OPRAH WINFREY",
                    360);
            bookService.create(book03);

            Author author03 = new Author("Vicki", "Robin", LocalDate.of(1945, 7, 6), null);
            authorService.create(author03);
            Author author04 = new Author("Joe", "Domínguez", LocalDate.of(1938, 2, 2), LocalDate.of(1997, 1, 11));
            authorService.create(author04);

            BookAuthor bookAuthor03 = new BookAuthor();
            book03.addBookAuthor(bookAuthor03);
            author03.addBookAuthor(bookAuthor03);
            bookAuthorService.create(bookAuthor03);

            BookAuthor bookAuthor04 = new BookAuthor();
            book03.addBookAuthor(bookAuthor04);
            author04.addBookAuthor(bookAuthor04);
            bookAuthorService.create(bookAuthor04);
        });
    }
}

# Test controller


Mediante el archivo `Controller test.ipynb`, que es un documento de jupyter notebook, corremos el primer test para probar el método `readAll` del controlador `BookController`. Ya que la respuesta de este método va a cambiar, a medida que modifiquemos el controlador, es que traemos una imagen de lo que nos devuelve la prueba. Para llamar al método `readAll` utilizamos el método `HTTP GET` como sigue. Si hacemos clic sobre la ruta también vamos a poder ver la respuesta en el navegador. En caso de que la respuesta sea de error debemos verificar que esté iniciado el servidor y como está configurado el método en el controlador.


> GET http://localhost:8080/api/books


![http_get_books_03](https://raw.githubusercontent.com/maq-miguel-quinteros/00_apuntes/main/03_java-python/API%20RESTful%20Java%20and%20SpringBoot/wip/img/http_get_books_03.png)
